In [102]:
from pydantic import BaseModel
from typing import List


class Activity(BaseModel):
    modulename: str
    moduleid: str
    title: str
    name: str
    directory: str
    content: str

class Section(BaseModel):
    title: str
    sectionid: str
    activities: List[Activity]

class Cours(BaseModel):
    title: str
    sections: List[Section]

cours = Cours(
    title="mon nouveau cours",
    sections=[
        Section(
            title="test",
            sectionid="4",
            activities=[
                Activity(
                    modulename="page",
                    moduleid="5",
                    title="Cours",
                    name="page_5",
                    directory="activities/page_5",
                    content="<div><h1>titre du cours</h1> <p>contenu du cours</p></div>"
                )
            ]
        )
    ]
)

In [120]:
# xml utils
import xml.etree.ElementTree as ET

def pprint_xml(xml):
    from xml.dom import minidom
    print('\n'.join([line for line in minidom.parseString(xml).toprettyxml(indent="  ").split('\n') if line.strip()]))# Clean up


def fill_moodle_activities(xml_content: str, cours: Cours) -> str:
    """
    Update Moodle backup XML with content from a Cours object.
    """
    # Parse the XML
    tree = ET.ElementTree(ET.fromstring(xml_content))
    root = tree.getroot()
    
    # Find the contents section
    contents = root.find(".//contents")
    
    # Clear and update activities and sections
    activities_elem = contents.find("activities")
    sections_elem = contents.find("sections")
    activities_elem.clear()
    sections_elem.clear()
    
    # Add new sections and activities
    for section in cours.sections:
        # Add section
        section_elem = ET.SubElement(sections_elem, "section")
        ET.SubElement(section_elem, "sectionid").text = section.sectionid
        ET.SubElement(section_elem, "title").text = section.title
        ET.SubElement(section_elem, "directory").text = f"sections/section_{section.sectionid}"
        
        # Add activities for this section
        for activity in section.activities:
            activity_elem = ET.SubElement(activities_elem, "activity")
            ET.SubElement(activity_elem, "moduleid").text = activity.moduleid
            ET.SubElement(activity_elem, "sectionid").text = section.sectionid
            ET.SubElement(activity_elem, "modulename").text = activity.modulename
            ET.SubElement(activity_elem, "title").text = activity.title
            ET.SubElement(activity_elem, "directory").text = activity.directory
    
    return ET.tostring(root, encoding='unicode', xml_declaration=True)



def fill_moodle_sections(xml_content: str, cours: Cours) -> str:
    """
    Update Moodle backup XML with content from a Cours object.
    """
    # Parse the XML
    tree = ET.ElementTree(ET.fromstring(xml_content))
    root = tree.getroot()
    
    # Find the contents section
    contents = root.find(".//contents")
    
    # Clear and update activities and sections
    activities_elem = contents.find("activities")
    sections_elem = contents.find("sections")
    activities_elem.clear()
    sections_elem.clear()
    
    # Add new sections and activities
    for section in cours.sections:
        # Add section
        section_elem = ET.SubElement(sections_elem, "section")
        ET.SubElement(section_elem, "sectionid").text = section.sectionid
        ET.SubElement(section_elem, "title").text = section.title
        ET.SubElement(section_elem, "directory").text = f"sections/section_{section.sectionid}"
        
        # Add activities for this section
        for activity in section.activities:
            activity_elem = ET.SubElement(activities_elem, "activity")
            ET.SubElement(activity_elem, "moduleid").text = activity.moduleid
            ET.SubElement(activity_elem, "sectionid").text = section.sectionid
            ET.SubElement(activity_elem, "modulename").text = activity.modulename
            ET.SubElement(activity_elem, "title").text = activity.title
            ET.SubElement(activity_elem, "directory").text = activity.directory
    
    return ET.tostring(root, encoding='unicode', xml_declaration=True)


def update_moodle_activity_module(xml_data: str, activity: Activity, sectionId: str) -> str:
    root = ET.fromstring(xml_data)
    # Update module values
    root.set("id", activity.moduleid)
    root.find("modulename").text = activity.modulename
    root.find("sectionid").text = sectionId
    
          
    
    return ET.tostring(root, encoding="unicode", xml_declaration=True)

def update_moodle_activity_page(xml_data: str, activity: Activity, sectionId: str) -> str:
    """
    Update the content of a page activity in Moodle backup XML.
    
    Args:
        xml_data (str): The original page activity XML content
        activity (Activity): Activity object containing the new content
        sectionId (str): Section ID for the activity
        
    Returns:
        str: Updated XML content
    """
    # Parse the XML
    root = ET.fromstring(xml_data)
    
    # Update activity attributes
    root.set('id', activity.moduleid)
    root.set('moduleid', activity.moduleid)
    root.set('modulename', activity.modulename)
    root.set('contextid', '20')
    
    # Find the page element and update its content
    page = root.find('page')
    if page is not None:
        # Update name/title
        name = page.find('name')
        if name is not None:
            name.text = activity.title
            
        # Update content
        content = page.find('content')
        if content is not None:
            content.text = activity.content
    
    # Convert back to string with XML declaration and proper encoding
    return ET.tostring(root, encoding='unicode', xml_declaration=True)






TEMP_FOLDER = 'temp'
MBZ_TEMPLATE = 'templates/backup-moodle-course.mbz'
PAGE_ACTIVITY_TEMPLATE_DIR = 'templates/activities/page'

import tarfile
import os
import uuid
import shutil

# Create a random folder name
random_folder = os.path.join(TEMP_FOLDER, str(uuid.uuid4()))




moodle_backup_xml_file = os.path.join(random_folder, 'moodle_backup.xml')
# Create temp directory and random subfolder if they don't exist
if not os.path.exists(TEMP_FOLDER):
    os.makedirs(TEMP_FOLDER)
if not os.path.exists(random_folder):
    os.makedirs(random_folder)

# Extract the template file
with tarfile.open(MBZ_TEMPLATE, 'r:gz') as tar_ref:
    tar_ref.extractall(random_folder)


# 1) UPDATE MOODLE_BACKUP.XML
with open(moodle_backup_xml_file, 'r') as f:
    moddle_backup_xml_text = f.read()
    moddle_backup_xml_text = fill_moodle_activities(moddle_backup_xml_text, cours)
    moddle_backup_xml_text = fill_moodle_sections(moddle_backup_xml_text, cours)
    # pprint_xml(updated_xml)
    with open(moodle_backup_xml_file, 'w') as f:
        f.write(moddle_backup_xml_text)






# 2) CREATE ACTIVITIES
activities_folder = os.path.join(random_folder, 'activities')
if os.path.exists(activities_folder):
    shutil.rmtree(activities_folder)
os.makedirs(activities_folder)

for section in cours.sections:
    for activity in section.activities:
        activity_dir = os.path.join(random_folder, activity.directory)
        os.makedirs(activity_dir)
        shutil.copytree(PAGE_ACTIVITY_TEMPLATE_DIR, activity_dir, dirs_exist_ok=True)

        # update 

        activity_module_xml_file_path = os.path.join(activity_dir, 'module.xml')
        with open(activity_module_xml_file_path, 'r') as f:
            activity_module_xml = f.read()
            # pprint_xml(activity_module_xml)
            updated_module = update_moodle_activity_module(activity_module_xml, activity, sectionId=section.sectionid)
            # pprint_xml(updated_module)
            with open(activity_module_xml_file_path, 'w') as f:
                f.write(updated_module)
            
        activity_page_xml_file_path = os.path.join(activity_dir, 'page.xml')
        with open(activity_page_xml_file_path, 'r') as f:
            activity_module_xml = f.read()
            # pprint_xml(activity_module_xml)
            updated_module = update_moodle_activity_page(activity_module_xml, activity, sectionId=section.sectionid)
            # pprint_xml(updated_module)
            with open(activity_page_xml_file_path, 'w') as f:
                f.write(updated_module)


# output_mbz = os.path.join(TEMP_FOLDER, f"backup-moodle-course.mbz")
output_mbz = os.path.join(TEMP_FOLDER, f"{cours.title}.mbz")
with tarfile.open(output_mbz, "w:gz") as tar:
    for item in os.listdir(random_folder):
        tar.add(os.path.join(random_folder, item), arcname=item)
# shutil.rmtree(random_folder)

